# Data processing - Level 0

In [ ]:
%load_ext autoreload
%autoreload 2
%matplotlib ipympl

In [ ]:
import logging
from helikite.constants import constants

logging.basicConfig(level=logging.INFO, format='%(message)s')

In [ ]:
from helikite.utils_exclude import get_flight_start_end

takeoff_time, landing_time = get_flight_start_end(constants.LEVEL1_FILEPATH)

### Load flight data

Cleaner detects instruments automatically based on the output schema and the files in the input folder. However, it is possible to specify the list of instruments to be used explicitly.

Output schema contains the list of reference instrument candidates, i.e. for ORACLES campaign it is `[flight_computer_v2, smart_tether, pops]`.
The first available instrument from the list will be used as a reference instrument, unless `reference_instrument` is specified explicitly.

If the reference instrument is different from the flight computer and its time is shifted relative to the real time, you can specify `reference_instrument_shift`.

In [ ]:
from helikite.classes.base import OutputSchemas
from helikite import Cleaner

cleaner = Cleaner(
    output_schema=OutputSchemas.from_name(constants.OUTPUT_SCHEMA),
    input_folder=constants.FLIGHT_DIRPATH,
    flight_date=constants.FLIGHT_DATE,
    flight=constants.FLIGHT,
    # instruments = [...]
    # reference_instrument=pops,
    # reference_instrument_shift="-50s",
)

In [ ]:
cleaner.set_time_as_index()
cleaner.flight_computer.df = cleaner.fill_missing_timestamps(cleaner.flight_computer.df, freq="1s", fill_method="ffill")
#cleaner.pops.df = cleaner.fill_missing_timestamps(cleaner.pops.df, freq="1s", fill_method="ffill")
cleaner.msems_inverted.df['scan_direction'] = cleaner.msems_scan.df['scan_direction']  # To have 0 / 1 values when changing scans --> for removal of duplicates
cleaner.data_corrections()
cleaner.set_pressure_column()

**If needed, manually shift instrument by X seconds**

In [ ]:
#cleaner.smart_tether.df = cleaner.smart_tether.df.shift(freq="-2s")       # ST
# cleaner.pops.df = cleaner.pops.df.shift(freq="-50s")                       # POPS
#cleaner.msems_readings.df = cleaner.msems_readings.df.shift(freq="-7s")   # mSEMS readings
#cleaner.msems_inverted.df = cleaner.msems_inverted.df.shift(freq="-14s")  # mSEMS inverted
#cleaner.msems_scan.df = cleaner.msems_scan.df.shift(freq="-14s")          # mSEMS scan
#cleaner.mcda.df = cleaner.mcda.df.shift(freq="-1s")                       # mCDA

### Flight time definition
Chose start and end of the flight.  
2 cases for ORACLES :  
- Start and stop at sledge height
- Start and stop at balloon height (addition of 3.5 m later in the processing)

In [ ]:
cleaner.define_flight_times()

In [ ]:
# print(cleaner.flight_computer.df.index.dtype)
# #print(cleaner.smart_tether.df.index.dtype)
# print(cleaner.pops.df.index.dtype)
# print(cleaner.msems_readings.df.index.dtype)
# print(cleaner.msems_inverted.df.index.dtype)
# print(cleaner.msems_scan.df.index.dtype)
# #print(cleaner.stap.df.index.dtype)
# #print(cleaner.mcda.df.index.dtype)

### Cross correlation
Time synchronisation based on the pressure measured by the different instruments.

In [ ]:
cleaner.correct_time_and_pressure(max_lag=180)

In [ ]:
cleaner.plot_pressure()

In [ ]:
# # MANUAL TIME CORRECTION
#
# cleaner.flight_computer.df = cleaner.flight_computer.df.shift(freq="2s")  # FC
# cleaner.smart_tether.df = cleaner.smart_tether.df.shift(freq="-6s")       # ST
# #cleaner.pops.df = cleaner.pops.df.shift(freq="64s")                       # POPS
# #cleaner.msems_readings.df = cleaner.msems_readings.df.shift(freq="-7s")   # mSEMS readings
# #cleaner.msems_inverted.df = cleaner.msems_inverted.df.shift(freq="-14s")  # mSEMS inverted
# #cleaner.msems_scan.df = cleaner.msems_scan.df.shift(freq="-14s")          # mSEMS scan
# #cleaner.mcda.df = cleaner.mcda.df.shift(freq="-1s")                       # mCDA

In [ ]:
# print(cleaner.df_corr.head())

### Dataframe creation
Removal of duplicates and merging instruments into one dataframe.

In [ ]:
cleaner.remove_duplicates()
cleaner.merge_instruments()
# cleaner.export_data()

### Overview plot of time synchronisation

In [ ]:
from helikite.instruments import msems_scan

save_path = constants.LEVEL0_DIRPATH / f"Level0_{constants.FLIGHT_BASENAME}_Flight_{constants.FLIGHT}_TimeSync.png"
cleaner.plot_time_sync(save_path, skip=[msems_scan])

## Level 0

In [ ]:
cleaner.master_df = cleaner.shift_msems_columns_by_90s(cleaner.master_df)

In [ ]:
cleaner.export_data(filename=constants.DATA_DIRPATH / "Processing" / "Level0" / f"level0_{constants.FLIGHT_BASENAME}")